# Lecture du Fichier de coordonnées de l'ISS

## [TODO] Introduction

https://spotthestation.nasa.gov/trajectory_data.cfm

After the header, ISS state vectors in the Mean of J2000 (J2K) reference frame are listed at four-minute intervals spanning a total length of 15 days. During reboosts (translation maneuvers), the state vectors are reported in two-second intervals. Each state vector lists the time in UTC; position X, Y, and Z in km; and velocity X, Y, and Z in km/s.

Orbit Ephemeris Message (OEM)

    https://nasa-public-data.s3.amazonaws.com/iss-coords/current/ISS_OEM/ISS.OEM_J2K_EPH.txt

In [1]:
# Affichage des 50 premières lignes du fichier
with open("ISS.OEM_J2K_EPH.txt") as file:
    file_lines = file.readlines()
    for line in file_lines[:50]:
        print(line, end='')

CCSDS_OEM_VERS = 2.0
CREATION_DATE  = 2023-03-24T22:55:31.604
ORIGINATOR     = NASA/JSC/FOD/TOPO

META_START
OBJECT_NAME          = ISS
OBJECT_ID            = 1998-067-A
CENTER_NAME          = Earth
REF_FRAME            = EME2000
TIME_SYSTEM          = UTC
START_TIME           = 2023-03-24T18:41:29.004
USEABLE_START_TIME   = 2023-03-24T18:41:29.004
USEABLE_STOP_TIME    = 2023-04-08T18:41:29.004
STOP_TIME            = 2023-04-08T18:41:29.004
META_STOP

COMMENT Source: This file was produced by the TOPO office within FOD at JSC.
COMMENT Units are in kg and m^2
COMMENT MASS=473832.00
COMMENT DRAG_AREA=1618.40
COMMENT DRAG_COEFF=2.50
COMMENT SOLAR_RAD_AREA=0.00
COMMENT SOLAR_RAD_COEFF=0.00
COMMENT Orbits start at the ascending node epoch
COMMENT ISS first asc. node: EPOCH = 2023-03-24T18:57:36.042 $ ORBIT = 2869 $ LAN(DEG) = -79.71025
COMMENT ISS last asc. node : EPOCH = 2023-04-08T18:04:09.079 $ ORBIT = 3101 $ LAN(DEG) = -155.16623
COMMENT Begin sequence of events
COMMENT TRAJECTORY EVENT

[TODO] Explications du contenu du fichier

## Différenciation des phases de poussée

In [2]:
import pandas as pd

# Sélection des lignes de commentaires
comments = [line for line in file_lines if line.startswith("COMMENT")]
# Stockage de l'index de la dernière ligne de commentaire en prévision de l'accès aux données
last_comment_index = file_lines.index(comments[-1])
# Stockage des données dans un DataFrame, à partir des lignes suivant le dernier commentaire
df = pd.read_csv("ISS.OEM_J2K_EPH.txt", skiprows=last_comment_index+1,
                 sep=" ", names=['datetime', 'x', 'y', 'z', 'vx', 'vy', 'vz'],
                 parse_dates=[0], infer_datetime_format=True)

df.datetime.diff().groupby(by=df.datetime.diff().rename('timedelta')).count().to_frame('count')

,count
timedelta,
0 days 00:00:00.727000,1
0 days 00:00:02,561
0 days 00:00:30.996000,1
0 days 00:01:29.004000,1
0 days 00:02:00,1
0 days 00:02:30,1
0 days 00:03:17.273000,1
0 days 00:03:30,1
0 days 00:04:00,5392


In [3]:
# Ajout d'une colonne spécifiant le nombre de minisecondes depuis la dernière position
df['delta'] = (df.diff().datetime.dt.total_seconds() * 1000).fillna(-1).astype(int)

In [4]:
# Affichage des deux enregistrements précédant et suivant les delta anormaux
for i in df[1:].query("delta != 2000 and delta != 240000").index:
    display(df[i-2:i+3].style.apply(lambda r:(r.size-1)*['']+['background-color: pink'
                                                              if i==r.name else ''], axis = 1))

,datetime,x,y,z,vx,vy,vz,delta
334,2023-03-25 16:57:29.004000,-32.306053,4395.313208,5171.615949,-7.328601,-1.739872,1.424871,240000
335,2023-03-25 17:01:29.004000,-1768.601204,3822.850551,5320.647421,-7.052094,-3.001539,-0.190606,240000
336,2023-03-25 17:02:00,-1986.063024,3727.498680,5311.481217,-6.978082,-3.150363,-0.400780,30996
337,2023-03-25 17:06:00,-3568.114525,2844.900706,5022.535203,-6.125024,-4.159732,-1.992457,240000
338,2023-03-25 17:10:00,-4890.252505,1755.091128,4366.711354,-4.825288,-4.866533,-3.439357,240000


,datetime,x,y,z,vx,vy,vz,delta
1308,2023-03-28 09:50:00,4225.585868,3696.122795,3822.263034,-5.938945,2.446453,4.179855,240000
1309,2023-03-28 09:54:00,2663.804724,4141.631605,4673.708001,-6.996374,1.243448,2.872057,240000
1310,2023-03-28 09:57:30,1133.934546,4284.770444,5140.436797,-7.505694,0.113394,1.552121,210000
1311,2023-03-28 10:01:30,-686.798056,4155.719089,5320.794144,-7.574373,-1.182310,-0.058431,240000
1312,2023-03-28 10:05:30,-2457.532806,3724.124451,5112.679275,-7.091499,-2.392425,-1.665360,240000


,datetime,x,y,z,vx,vy,vz,delta
2131,2023-03-30 16:41:30,-6137.648570,-1626.790188,-2423.180452,3.270230,-4.396864,-5.352619,240000
2132,2023-03-30 16:45:30,-5139.257690,-2610.107246,-3603.845410,4.998402,-3.747663,-4.426360,240000
2133,2023-03-30 16:48:00,-4319.943913,-3132.459109,-4213.222995,5.899639,-3.200577,-3.679449,150000
2134,2023-03-30 16:48:02,-4308.133701,-3138.852325,-4220.571185,5.910569,-3.192636,-3.668737,2000
2135,2023-03-30 16:48:04,-4296.301659,-3145.229641,-4227.897932,5.921468,-3.184678,-3.658007,2000


,datetime,x,y,z,vx,vy,vz,delta
2693,2023-03-30 17:06:40,3672.060857,-3670.315479,-4397.132683,6.433781,2.368560,3.393198,2000
2694,2023-03-30 17:06:42,3684.919130,-3665.569085,-4390.335149,6.424486,2.377831,3.404334,2000
2695,2023-03-30 17:06:42.727000,3689.588500,-3663.839178,-4387.858727,6.421099,2.381198,3.408378,727
2696,2023-03-30 17:10:42.727000,5078.505471,-2966.683496,-3420.539266,5.082637,3.393317,4.603714,240000
2697,2023-03-30 17:14:42.727000,6099.588262,-2054.672037,-2204.794384,3.374275,4.160700,5.465858,240000


,datetime,x,y,z,vx,vy,vz,delta
3412,2023-04-01 16:54:42.727000,-2080.494653,-3794.968955,-5245.695845,7.162813,-2.472585,-1.057779,240000
3413,2023-04-01 16:58:42.727000,-306.945418,-4243.965230,-5306.216820,7.526993,-1.246446,0.556363,240000
3414,2023-04-01 17:02:00,1173.305736,-4383.969979,-5067.110137,7.418617,-0.167121,1.857804,197273
3415,2023-04-01 17:06:00,2889.823903,-4265.048202,-4442.865063,6.798774,1.152257,3.312707,240000
3416,2023-04-01 17:10:00,4397.201959,-3837.486571,-3496.201205,5.686178,2.389316,4.528283,240000


,datetime,x,y,z,vx,vy,vz,delta
4845,2023-04-05 16:26:00,-4153.876973,4837.611360,2339.249979,-5.141748,-1.801002,-5.392888,240000
4846,2023-04-05 16:30:00,-5221.487976,4234.342581,975.274856,-3.700531,-3.195519,-5.903839,240000
4847,2023-04-05 16:32:00,-5616.403780,3813.295424,260.031477,-2.871359,-3.811194,-5.998614,120000
4848,2023-04-05 16:36:00,-6092.515827,2770.816039,-1171.563371,-1.072191,-4.822882,-5.858194,240000
4849,2023-04-05 16:40:00,-6124.940211,1526.543421,-2517.621273,0.803250,-5.482567,-5.290338,240000


,datetime,x,y,z,vx,vy,vz,delta
5958,2023-04-08 18:36:00,95.433167,5138.767287,4438.176112,-6.244148,2.971065,-3.303597,240000
5959,2023-04-08 18:40:00,-1388.411239,5656.152123,3493.021281,-6.045635,1.313880,-4.524614,240000
5960,2023-04-08 18:41:29.004000,-1918.603538,5744.419605,3073.354355,-5.858246,0.667872,-4.897754,89004


Les écarts non-standards débutent ou terminent les poussées, ou sont sporatiques dans le jeu de données.
On retiendra donc uniquement les intervales de 2 secondes pour représenter les poussées.

In [5]:
# poussée

In [6]:
import numpy as np

# Les enregistrements datant de deux secondes après leur précédent sont marqués en tant que poussée
df['on_thrust'] = np.isclose(df.diff().datetime.dt.total_seconds(), 2)

# Initialisation des valeurs d'épisode de poussée
current_state = df.iloc[0]['on_thrust']
current_tid = 0

def thrust_episode(on_thrust):
    """Fonction pour le parours de proche en proche des états de poussée pour la définition d'épisodes."""
    global current_state, current_tid
    # Si l'état actuel est différent du dernier état enregistré
    if on_thrust != current_state:
        # Changement d'état et d'identifiant d'épisode.
        current_state = on_thrust
        current_tid += 1
    # Renvoi de l'indentifiant actuel
    return current_tid

# Un épsiode est une séquence contiguë d'états de poussée
df['thrust_episode'] = df['on_thrust'].apply(thrust_episode)

# Décompte des épisodes identifiés
df.groupby('thrust_episode').count()[['on_thrust']]

,on_thrust
thrust_episode,
0,2134
1,561
2,3266


## Classe pour la bibliothèque

Pour permettre l'utilisation simple des données de ce ficiher, nous définissons une classe `ISS_Position` pour notre bibliothèque. Une instance de cette classe permet d'accéder directement aux métadonnées, commentaires et données contenus dans le fichier par des méthodes respectivement nommées `get_metadata()`, `get_comments()` et `get_data()`.

In [7]:
class ISS_Position:
    """Classe d'extraction des données de position de l'ISS."""
    def __init__(self, filename="ISS.OEM_J2K_EPH.txt"):
        """Reçoit en paramètre le nom du fichier de données. Utilise "ISS.OEM_J2K_EPH.txt" par défaut."""
        self.__parse_source(filename)
        
    def __parse_source(self, filename):
        """Analyse syntaxique du fichier."""
        with open(filename) as file:
            source_lines = file.readlines()
        # Extraction des métadonnées
        self.__parse_source_meta(source_lines)
        # Extraction des commentaires
        self.__parse_source_comments(source_lines)
        # Extraction des données
        self.__parse_data(filename)
    
    def __parse_source_meta(self, source_lines):
        """Extraction des métadonnées du fichier."""
        # Détection des lignes de métadonnées
        meta = slice(source_lines.index('META_START\n')+1, source_lines.index('META_STOP\n'))
        # Enregistrement des métadonnées dans un dictionnaire
        self.meta = {key.strip(): pd.to_datetime(value.strip()) if key.strip().endswith('_TIME') else value.strip()
                     for line in source_lines[meta]
                     for (key, value) in [line.split('=', 1)]
                    }
    
    def __parse_source_comments(self, source_lines):
        """Extraction des commentaires du fichier."""
        # Sélection des lignes de commentaires
        comments = [line for line in source_lines if line.startswith("COMMENT")]
        # Stockage de l'index de la dernière ligne de commentaire en prévision de l'accès aux données
        self.__last_comment_index = source_lines.index(comments[-1])
        # Enregistrement des commentaires dans une liste
        self.comments = [comment[7:].strip() for comment in comments]
    
    def __parse_data(self, filename):
        """Extraction des données du fichier."""
        # Enregistrement des données dans un DataFrame, à partir des lignes suivant le dernier commentaire
        self.data = pd.read_csv(filename, skiprows=self.__last_comment_index+1,
                                sep=" ", names=['datetime', 'x', 'y', 'z', 'vx', 'vy', 'vz'],
                                parse_dates=[0], infer_datetime_format=True)
        # Ajoute les annotations de poussée
        self.__add_thrust_metadata()
        
    def __add_thrust_metadata(self):
        """Ajout des annotation de poussé et épisodes de poussée."""
        # Les enregistrements datant de deux secondes après leur précédent sont marqués en tant que poussée
        self.data['on_thrust'] = np.isclose(self.data.diff().datetime.dt.total_seconds(), 2)
        # Initialisation des valeurs d'épisode de poussée
        self.__current_state = self.data.iloc[0]['on_thrust']
        self.__current_tid = 0
        # Un épsiode est une séquence contiguë d'état de poussée
        self.data['thrust_episode'] = self.data['on_thrust'].apply(self.__thrust_episode)
        # Suppression des marqueurs inutiles
        del self.__current_state, self.__current_tid

    def __thrust_episode(self, on_thrust):
        """Méthode pour le parours de proche en proche des états de poussée pour la définition d'épisodes."""
        # Si l'état actuel est différent du dernier état enregistré
        if on_thrust != self.__current_state:
            # Changement d'état et d'identifiant d'épisode.
            self.__current_state = on_thrust
            self.__current_tid += 1
        # Renvoi de l'indentifiant actuel
        return self.__current_tid


    def get_metadata(self):
        # Retourne les métadonnées en utilisant un DataFrame (pour son rendu élégant à l'affichage)
        return pd.DataFrame(self.meta.values(), self.meta.keys(), ['Metadata'])
    
    def get_comments(self):
        # Retourne les commentaires en tant que bloc de texte avec retours à la ligne
        return '\n'.join(self.comments)
    
    def get_data(self):
        # Retourne le DataFrame des données
        return self.data

## Test de la classe

Pour tester notre classe, nous l'instancions et appellons ses méthodes publiques.

In [8]:
# Instanciation de la classe
iss_position = ISS_Position()

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [9]:
# Accès aux métadonnées
iss_position.get_metadata()

,Metadata
OBJECT_NAME,ISS
OBJECT_ID,1998-067-A
CENTER_NAME,Earth
REF_FRAME,EME2000
TIME_SYSTEM,UTC
START_TIME,2023-03-24 18:41:29.004000
USEABLE_START_TIME,2023-03-24 18:41:29.004000
USEABLE_STOP_TIME,2023-04-08 18:41:29.004000
STOP_TIME,2023-04-08 18:41:29.004000


In [10]:
# Accès aux commentaires
print(iss_position.get_comments())

Source: This file was produced by the TOPO office within FOD at JSC.
Units are in kg and m^2
MASS=473832.00
DRAG_AREA=1618.40
DRAG_COEFF=2.50
SOLAR_RAD_AREA=0.00
SOLAR_RAD_COEFF=0.00
Orbits start at the ascending node epoch
ISS first asc. node: EPOCH = 2023-03-24T18:57:36.042 $ ORBIT = 2869 $ LAN(DEG) = -79.71025
ISS last asc. node : EPOCH = 2023-04-08T18:04:09.079 $ ORBIT = 3101 $ LAN(DEG) = -155.16623
Begin sequence of events
TRAJECTORY EVENT SUMMARY:

|       EVENT        |       TIG        | ORB |   DV    |   HA    |   HP    |
|                    |       GMT        |     |   M/S   |   KM    |   KM    |
|                    |                  |     |  (F/S)  |  (NM)   |  (NM)   |
68S Undocking         087:09:57:30.000             0.0     426.1     408.5
(0.0)   (230.1)   (220.6)

GMT 089 Cygnus Reb    089:16:48:00.000             1.1     425.4     408.5
(3.6)   (229.7)   (220.6)

End sequence of events


In [11]:
# Accès aux données
iss_position.get_data()

,datetime,x,y,z,vx,vy,vz,r,theta,phi,on_thrust,thrust_episode
0,2023-03-24 18:41:29.004,4482.285420,-1945.879050,-4735.454317,5.092194,4.987269,2.771175,4885.958699,2.892744,-0.409575,False,0
1,2023-03-24 18:45:29.004,5527.582022,-693.041910,-3906.729727,3.565402,5.389738,4.093092,5570.508358,2.348049,-0.124728,False,0
2,2023-03-24 18:49:29.004,6172.791603,609.934836,-2794.459786,1.778186,5.402386,5.119574,6202.627041,2.038153,0.098491,False,0
3,2023-03-24 18:53:29.004,6370.538918,1868.675170,-1479.074801,-0.140821,5.023142,5.775192,6638.842814,1.795473,0.285327,False,0
4,2023-03-24 18:57:29.004,6105.797591,2991.735742,-56.034355,-2.052317,4.278463,6.011005,6799.352244,1.579038,0.455602,False,0
...,...,...,...,...,...,...,...,...,...,...,...,...
5956,2023-04-08 18:28:00.000,2934.956126,3047.441288,5312.015084,-5.297560,5.528618,-0.248731,4231.569198,NaN,0.804199,False,2
5957,2023-04-08 18:32:00.000,1572.344931,4247.491496,5059.514540,-5.988345,4.410671,-1.842741,4529.736428,NaN,1.216256,False,2
5958,2023-04-08 18:36:00.000,95.433167,5138.767287,4438.176112,-6.244148,2.971065,-3.303597,5140.085105,0.528739,1.552227,False,2
5959,2023-04-08 18:40:00.000,-1388.411239,5656.152123,3493.021281,-6.045635,1.313880,-4.524614,5824.365684,0.927638,1.811506,False,2


In [12]:
# Décompte des intervalles pour dénombrer les poussées
x = pd.Series(iss_position.get_data().query('on_thrust').index).diff()
x.groupby(x).count()

1.0    560
dtype: int64

Une seule poussée (de 1120s ≈ 19min) dans le jeu de données.